# GMM for anomaly detection

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline



from sklearn.model_selection import train_test_split
from sklearn import mixture
from sklearn.mixture import GaussianMixture as GMM
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn import metrics


#  Get DK data

In [ ]:
import zipfile
with zipfile.ZipFile('hackathon_kpis_anonymised.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

# Gaussian Mixture Models EM

In [ ]:
df = pd.read_csv('hackathon_kpis_anonymised.csv')

In [ ]:
df

In [ ]:
df = df.sort_values(by = 'timestamp')

In [ ]:
df.head()

In [ ]:
nan_values = df.isna()
nan_columns = nan_values.any()

columns_with_nan = df.columns[nan_columns].tolist()
print(columns_with_nan)

In [ ]:
df = df.fillna(0.)

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
cell_occurrences = pd.DataFrame(df.cell_name.value_counts())
cell_occurrences.columns = ['count']

In [ ]:
cell_occurrences

In [ ]:
cell_occurrences['count'].hist()

### Optimal clusters

In [ ]:
# full dataset

XX = np.array(df[['avail_period_duration', 'bandwidth',
       'num_voice_attempts', 'num_data_attempts', 'voice_tot_failure_rate',
       'data_tot_failure_rate', 'unavail_unplan_rate', 'unavail_total_rate',
       'voice_setup_failure_rate', 'voice_drop_rate',
       'data_setup_failure_rate', 'data_drop_rate', 'throughput_rate',
       'ho_failure_rate']].values)

In [ ]:
XX.shape

In [ ]:
# selecting n columns of the dataset

n = 14
X = XX[:,:n]

In [ ]:
# splitting the dataset into train-test by using dates

nobs = df[df['timestamp']>='2020-11'].shape[0]

X_train  = X[:-nobs,:]
X_test = X[-nobs:,:]

# getting the train-test percentage

print('The total dataset consists of',X.shape[0],'rows')
print('The training dataset consists of',X_train.shape[0],'rows')
print('The test dataset consists of',X_test.shape[0],'rows')
print(np.round(X_train.shape[0]/X.shape[0]*100), '-',np.round(X_test.shape[0]/X.shape[0]*100),'percent split')

###  Setting parameter on split

- If you want to train on the whole dataset set **train_test = False**
- If you want to plit the dataset in training and testing set **train_test = True**
<br>


In [ ]:
#Parameter  to set
train_test = False

In [ ]:
# random sample for finding best cluster number


if  train_test == True:
    size_sample = 10000
    randomly_sampled = np.random.choice(X_train.shape[0], size=size_sample, replace=False)

    X_sample = X[randomly_sampled,:]
else:
    size_sample = 10000
    randomly_sampled = np.random.choice(X.shape[0], size=size_sample, replace=False)

    X_sample = X[randomly_sampled,:]

    

In [ ]:
rcParams['figure.figsize'] = 8,3

if train_test  ==  True:
    n_components = np.arange(1, 12)
    models = [GMM(n, covariance_type='full', random_state=0).fit(X_sample)
              for n in n_components]

    plt.plot(n_components, [m.bic(X_train) for m in models], label='BIC')
    plt.plot(n_components, [m.aic(X_train) for m in models], label='AIC')
    plt.legend(loc='best')
    plt.xlabel('n_components');
else:
    n_components = np.arange(1, 12)
    models = [GMM(n, covariance_type='full', random_state=0).fit(X_sample)
              for n in n_components]

    plt.plot(n_components, [m.bic(X) for m in models], label='BIC')
    plt.plot(n_components, [m.aic(X) for m in models], label='AIC')
    plt.legend(loc='best')
    plt.xlabel('n_components');
    

In [ ]:
n_components = 2

In [ ]:
if train_test  ==  True:
    gmm=GMM(n_components, n_init=1).fit(X_train) 
    labels=gmm.predict(X_test)
    scores = gmm.predict_proba(X_test)
else:
    gmm=GMM(n_components, n_init=1).fit(X) 
    labels=gmm.predict(X)
    scores = gmm.predict_proba(X)

In [ ]:
#eps = 1/n_components +0.1/n_components
eps=0.55

In [ ]:
if train_test:
    indices_test = [i for i, x in enumerate(scores.max(axis=1)<eps) if x == True]
    df_test = df[df['timestamp']>='2020-11'].reset_index().drop('index',1)
    df_test['anomaly'] = np.where(df_test.index.isin(indices_test), 1, 0)
    df_test =  df_test.set_index('timestamp')
    anomaly_dataset = df_test[df_test['anomaly']==1]
    print('The model has found',df_test[df_test['anomaly']==1].shape[0],'anomalies')
    print('associated to the following cells:')
    print(df_test[df_test['anomaly']==1]['cell_name'].value_counts())

else:
    indices = [i for i, x in enumerate(scores.max(axis=1)<eps) if x == True]
    df_tot = df.reset_index().drop('index',1)
    df_tot['anomaly'] = np.where(df.index.isin(indices), 1, 0)
    df_tot =  df_tot.set_index('timestamp') 
    anomaly_dataset = df_tot[df_tot['anomaly']==1]
    print('The model has found',df_tot[df_tot['anomaly']==1].shape[0],'anomalies')
    print('associated to the following cells:')
    print(df_tot[df_tot['anomaly']==1]['cell_name'].value_counts())

## List of all anomalies identified with GMM using 2 components and eps for thresholding probabilities

In [ ]:
anomaly_dataset

In [ ]:
anomaly_df = pd.DataFrame(anomaly_dataset['cell_name'].value_counts())
anomaly_df = anomaly_df.reset_index()
anomaly_df.columns = ['cell_name','count']
anomaly_df['site'] = anomaly_df['cell_name'].apply(lambda x: x[:2])

In [ ]:
site_cnt = anomaly_df['site'].value_counts().to_dict()

In [ ]:
anomaly_df['site_cnt'] = anomaly_df['site'].apply(lambda x: int(site_cnt[x]))

In [ ]:
anomaly_df = anomaly_df.sort_values('cell_name')

In [ ]:
anomaly_df

In [ ]:
anomaly_df[anomaly_df['site'] =='00']['cell_name'].unique()

## Example of anomalous behaviour in a cell

In [ ]:
df_plot = {}

if train_test:
    for i in  anomaly_dataset['cell_name'].value_counts().index:
        df_plot[i] = df_test[df_test['cell_name']== i]
else:
    for i in  anomaly_dataset['cell_name'].value_counts().index:
        df_plot[i] = df_tot[df_tot['cell_name']== i]
    

In [ ]:
if train_test:
    cell_most_anomalous_events  = anomaly_dataset['cell_name'].value_counts().index[0]
else:
    cell_most_anomalous_events = anomaly_dataset['cell_name'].value_counts().index[0]

In [ ]:
#checking how many timestamps we have for the cell with most number of anomlies:
print('The cell with most anomalous events has\n\n',cell_occurrences.loc[cell_most_anomalous_events], '\n\ntimestamps')


In [ ]:
anomaly_dataset[anomaly_dataset['cell_name'] == cell_most_anomalous_events]

## Some plots

In [ ]:
df_plot[cell_most_anomalous_events].plot(subplots = True, figsize = (20,20));


In [ ]:
df_plot['02_31Q'].plot(subplots = True, figsize = (20,20));